In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import wandb
import os
import random
from time import time

from lib.data import EpisodeDataset, EpisodeDataset_Unmodified
from lib.neural_networks import FullyConnected, FullyConnected3, ConvNet

In [11]:
WIDTH = 1
OBS_SIZE = 30 #46 is standard, 15 is minimal
HIDDEN_SIZE = 1024

In [51]:
TRAIN_PATH_1 = '/data/hmead/Episode Data/Random Data/30 TPS/3v3 Train'
TRAIN_PATH_2 = '/data/hmead/Episode Data/Nexto Data/30 TPS/3v3 Train'

train_dataset = EpisodeDataset([TRAIN_PATH_1, TRAIN_PATH_2], width=WIDTH, include_change=True, corrupt=True)

print(len(train_dataset))

Loading Episode Data
Finished Loading Episode Data
311700


In [52]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=32, pin_memory=True)

In [7]:
TEST_PATH = '/data/hmead/Episode Data/Human Example Data/30 TPS'
#TEST_PATH = '/data/hmead/Episode Data/Human Replays'

test_dataset = EpisodeDataset([TEST_PATH], width=WIDTH, include_change=True, corrupt=False)
print(len(test_dataset))

Loading Episode Data
Finished Loading Episode Data
129


In [8]:
Test_Data_Loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=16)

In [9]:
#model = ConvNet(obs_size=OBS_SIZE, obs_width=WIDTH, conv_number=10, hidden_size=HIDDEN_SIZE)
model = FullyConnected(obs_size=OBS_SIZE, obs_width=WIDTH, hidden_size=HIDDEN_SIZE)

gpumodel = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.0001)
#scheduler = ExponentialLR(optimizer, gamma = 0.63)

In [10]:
gpumodel.load_state_dict(torch.load('trained_networks/pretty-galaxy-3.pt'))

FileNotFoundError: [Errno 2] No such file or directory: 'trained_networks/pretty-galaxy-3.pt'

In [56]:
def test_model(gpumodel, Test_Data_Loader):
    gpumodel.eval()

    total_loss = 0
    obs_count = 0
    total_correct = torch.zeros(9).cuda()

    for _obs, _act, _add_data in Test_Data_Loader:
        # EVALUATION
        obs = _obs[0].cuda()
        act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
        add_data = _add_data[0].long()

        if obs.shape[0] == 0:
            continue

        with torch.no_grad():
            y_pred = model(obs)

        pred_act = torch.zeros(obs.shape[0], 9).cuda()

        loss = 0
        for i in range(9):
            loss += criterion(y_pred[i], act[:, i])

            pred_act[:, i] = torch.argmax(y_pred[i], dim=1)


        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)

        correct = act == pred_act
        total_correct += torch.sum(correct, dim=0)

    return total_loss / obs_count, total_correct / obs_count

In [57]:
wandb.init(project="Inverse Dynamics Model Retest", entity="harrymead")

wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 10,
  "batch_size": 10
}

accuracy/boost,▁▄▆▆▆▆▇▇▇▇▇▇▇▇▇██▇█████▇████████████████
accuracy/handbrake,▁▃▄▄▅▅▆▆▆▆▆▆▇▇▇▇█▇█▇▇▇█▇████████▇███████
accuracy/jump,▁▃▅▅▆▆▆▇▇▇▇▇▇▇█▇███████▇████████████████
accuracy/on_ground,▁▄▆▆▆▇▇▇▇▇▇▇▇█▇▇█▇██████████████████▇███
accuracy/pitch,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█▇▇████████████████████
accuracy/roll,▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇█████████████████████████
accuracy/steer,▁▄▅▆▆▇▇▇▇▇▇▇▇▇██████████████████████████
accuracy/throttle,▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇████████████████████████
accuracy/yaw,▁▄▅▆▆▇▇▇▇▇▇▇▇▇██████████████████████████
loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy/boost,▂▆▂▂▇▃▃▅▅▄▄▆▄▄▂▇▄▄▆▃▇█▇▄▄▅▂▅▅▄▄▁▁▇▆▃▅▄▅▇


In [58]:
BUFFER_SIZE = 5

obs_buffer = []
act_buffer = []
add_buffer = []

gpumodel.train()

for epoch in range(30):
    batch = 0
    total_loss = 0
    obs_count = 0
    total_correct = torch.zeros(9).cuda()

    for _obs, _act, _add_data in Train_Data_Loader:
        # OBS BATCHING 
        obs = _obs[0].cuda()
        act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
        add_data = _add_data[0]

        if obs.shape[0] == 0:
            continue

        obs_buffer.append(obs)
        act_buffer.append(act)
        add_buffer.append(add_data)

        if len(obs_buffer) < BUFFER_SIZE:
            continue

        obs = torch.cat(obs_buffer)
        act = torch.cat(act_buffer)
        add_data = torch.cat(add_buffer)

        obs_buffer = []
        act_buffer = []
        add_buffer = []

        # TRAINING

        optimizer.zero_grad()

        y_pred = model(obs)
        pred_act = torch.zeros(obs.shape[0], 9).cuda()

        loss = 0
        for i in range(9):
            loss += criterion(y_pred[i], act[:, i])

            pred_act[:, i] = torch.argmax(y_pred[i], dim=1)

        loss.backward()
        optimizer.step()


        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)

        correct = act == pred_act
        total_correct += torch.sum(correct, dim=0)


        batch += 1
        if batch % 1000 == 0:
            idv_accuracy = total_correct/obs_count

            if (batch - 1000) % 5000 == 0:
                start_test = time()
                test_loss, test_accuracy = test_model(gpumodel, Test_Data_Loader)
                gpumodel.train()

            print(f'Epoch {epoch + 1} - Loss: {total_loss / obs_count}, Correct: {idv_accuracy}')
            wandb.log({"loss": total_loss/obs_count, "accuracy/throttle": idv_accuracy[0], "accuracy/steer": idv_accuracy[1], "accuracy/pitch": idv_accuracy[2], 
                        "accuracy/yaw": idv_accuracy[3], "accuracy/roll": idv_accuracy[4], "accuracy/jump": idv_accuracy[5], "accuracy/boost": idv_accuracy[6],
                        "accuracy/handbrake": idv_accuracy[7], "accuracy/on_ground": idv_accuracy[8],
                        
                        "test_loss": test_loss, "test_accuracy/throttle": test_accuracy[0], "test_accuracy/steer": test_accuracy[1], "test_accuracy/pitch": test_accuracy[2], 
                        "test_accuracy/yaw": test_accuracy[3], "test_accuracy/roll": test_accuracy[4], "test_accuracy/jump": test_accuracy[5], "test_accuracy/boost": test_accuracy[6],
                        "test_accuracy/handbrake": test_accuracy[7], "test_accuracy/on_ground": test_accuracy[8],})

            total_loss = 0
            obs_count = 0
            total_correct = torch.zeros(9).cuda()
    
    #scheduler.step()
torch.save(gpumodel.state_dict(), f"trained_networks/{wandb.run.name} (30 epochs).pt")

Epoch 1 - Loss: 3.433783028074102, Correct: tensor([0.7986, 0.7069, 0.8294, 0.7071, 0.8518, 0.9356, 0.8666, 0.9663, 0.9307],
       device='cuda:0')
Epoch 1 - Loss: 2.4496771709099816, Correct: tensor([0.8939, 0.7692, 0.8795, 0.7692, 0.8940, 0.9437, 0.9445, 0.9680, 0.9794],
       device='cuda:0')
Epoch 1 - Loss: 2.1461982623776574, Correct: tensor([0.9143, 0.7885, 0.8972, 0.7884, 0.9045, 0.9484, 0.9589, 0.9683, 0.9850],
       device='cuda:0')
Epoch 1 - Loss: 2.0067944540708202, Correct: tensor([0.9238, 0.8003, 0.9050, 0.8002, 0.9065, 0.9490, 0.9621, 0.9690, 0.9868],
       device='cuda:0')
Epoch 1 - Loss: 1.8918020393352946, Correct: tensor([0.9295, 0.8067, 0.9090, 0.8067, 0.9096, 0.9511, 0.9666, 0.9693, 0.9888],
       device='cuda:0')
Epoch 1 - Loss: 1.8195710332593886, Correct: tensor([0.9337, 0.8136, 0.9126, 0.8136, 0.9109, 0.9524, 0.9689, 0.9691, 0.9890],
       device='cuda:0')
Epoch 1 - Loss: 1.7500547317995452, Correct: tensor([0.9370, 0.8198, 0.9136, 0.8199, 0.9127, 0.9543, 

wandb: Network error (ReadTimeout), entering retry loop.


Epoch 21 - Loss: 0.5148070853460907, Correct: tensor([0.9775, 0.9633, 0.9627, 0.9629, 0.9728, 0.9794, 0.9893, 0.9842, 0.9979],
       device='cuda:0')
Epoch 21 - Loss: 0.5170060285361735, Correct: tensor([0.9774, 0.9630, 0.9628, 0.9630, 0.9722, 0.9791, 0.9896, 0.9845, 0.9982],
       device='cuda:0')
Epoch 21 - Loss: 0.5007200301750735, Correct: tensor([0.9778, 0.9639, 0.9638, 0.9641, 0.9730, 0.9795, 0.9894, 0.9846, 0.9983],
       device='cuda:0')
Epoch 21 - Loss: 0.508131568636185, Correct: tensor([0.9779, 0.9642, 0.9626, 0.9641, 0.9723, 0.9784, 0.9891, 0.9849, 0.9985],
       device='cuda:0')
Epoch 21 - Loss: 0.5212046018206985, Correct: tensor([0.9774, 0.9623, 0.9628, 0.9624, 0.9724, 0.9782, 0.9891, 0.9842, 0.9983],
       device='cuda:0')
Epoch 21 - Loss: 0.505102829985943, Correct: tensor([0.9780, 0.9642, 0.9631, 0.9642, 0.9724, 0.9787, 0.9893, 0.9853, 0.9985],
       device='cuda:0')
Epoch 21 - Loss: 0.5084765407590508, Correct: tensor([0.9775, 0.9638, 0.9634, 0.9639, 0.9725, 0.

In [12]:
conf_mtx = torch.zeros((8,3,3)).cuda()
CONF_MAT = True

gpumodel.eval()

total_loss = 0
obs_count = 0
total_correct = torch.zeros(9).cuda()

correct_on_ground = torch.zeros(9).cuda()
total_on_ground = 0

for _obs, _act, _add_data in Test_Data_Loader:
    # EVALUATION
    obs = _obs[0].cuda()
    act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
    add_data = _add_data[0].long()

    if obs.shape[0] == 0:
        continue

    with torch.no_grad():
        y_pred = model(obs)

    y_pred = model(obs)
    pred_act = torch.zeros(obs.shape[0], 9).cuda()

    loss = 0
    for i in range(9):
        loss += criterion(y_pred[i], act[:, i])

        pred_act[:, i] = torch.argmax(y_pred[i], dim=1)


    # LOGGING
    obs_count += len(obs)
    total_loss += loss.item() * len(obs)

    correct = act == pred_act
    total_correct += torch.sum(correct, dim=0)

    on_ground = pred_act[:,8]

    #print(correct)
    #print(on_ground.bool())

    total_on_ground += torch.sum(on_ground == 1).item()


    correct_on_ground += torch.sum(correct[on_ground == 1], dim=0)

    if CONF_MAT:
        for i in range(obs.shape[0]):
            for j in range(8):
                if on_ground[i] == 1:
                    conf_mtx[j, act[i, j], pred_act[i, j].long()] += 1

    

print(f'Loss: {total_loss / obs_count}, Correct: {total_correct / obs_count}, Correct On Ground: {correct_on_ground / total_on_ground}, On Ground: {total_on_ground / obs_count}')

np.set_printoptions(precision=4, suppress=True)
print((conf_mtx).cpu().numpy())

Loss: 26.654266357421875, Correct: tensor([0.9120, 0.8223, 0.8248, 0.4793, 0.8872, 0.9554, 0.9545, 0.9802, 0.2843],
       device='cuda:0'), Correct On Ground: tensor([0.9169, 0.8199, 0.9977, 0.4163, 1.0000, 0.9867, 0.9555, 0.9723, 0.0000],
       device='cuda:0'), On Ground: 0.715702479338843
[[[ 183.    6.    6.]
  [  30.  359.   33.]
  [   5.   64. 1046.]]

 [[ 297.   45.    3.]
  [  67.  608.  118.]
  [  10.   69.  515.]]

 [[   0.    0.    0.]
  [   1. 1728.    3.]
  [   0.    0.    0.]]

 [[   0.    0.    0.]
  [ 368.  721.  643.]
  [   0.    0.    0.]]

 [[   0.    0.    0.]
  [   0. 1732.    0.]
  [   0.    0.    0.]]

 [[1709.   23.    0.]
  [   0.    0.    0.]
  [   0.    0.    0.]]

 [[1380.   50.    0.]
  [  27.  275.    0.]
  [   0.    0.    0.]]

 [[1684.   48.    0.]
  [   0.    0.    0.]
  [   0.    0.    0.]]]
